# Реализация игры Grid World с помощью Value-iteration

--------------------------------------------------

## Правила игры
![Img](https://raw.githubusercontent.com/DPritykin/Control-Theory-Course/main/Term%202%20Nonlinear%20Dynamics%20and%20Control/Images/gridworld.png)

<font size="3"> Агент начинает игру в произвольном свободном положении(на примере слева снизу поля) и заканчивает в ячейке с  +1 ("сектор приз") либо -1 ("сектор штраф"). 
    На каждом шагу он может двигаться вертикально или горизонтально, но не может проходить через препятствия или уходить за пределы игрового поля. 
    Подразумевается, что действие на каждом шаге детерминировано, то есть если агент выбрал действие "переместиться в клетку справа", то именно такое действие и реализуется (нет случайных факторов, которые могут повлиять на выполнение команды).</font>

--------------------------------------------------

## Поле
<font size="3">Зададим переменные, определяющие поле для игры</font>

In [1]:
import numpy as np

BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3)
LOSE_STATE = (1, 3)
START = (2, 0)
OBSTACLES = [(1, 1), (1, 2)]
DETERMINISTIC = True

<font size="3">Класс State определяет параметры текущего состояния, а также возвращает следущее состояние в зависимости от выбранного действия</font>

In [2]:
class State:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.obstacles = OBSTACLES
        for obs in self.obstacles:
            self.board[obs[0], obs[1]] = -1
        self.action_dict = {
            "up": (-1, 0),
            "down": (1, 0),
            "left": (0, -1),
            "right": (0, 1),
        }
        self.state = state
        self.is_end = False
        self.determine = DETERMINISTIC

    # Определяет награду согласно текущему состоянию
    def give_reward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0

    # Определяет, является ли текущее состояние state терминальным
    def update_end(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.is_end = True
    
    # Проверяет, лежит ли состояние state в границах игрового поля
    def is_within_bounds(self, state):
        return state[0] >= 0 and state[0] < self.board.shape[0] and \
               state[1] >= 0 and state[1] < self.board.shape[1]

    # Возвращает валидное следущее состояние в зависимости от выбранного действия action
    def next_position(self, action):
        if self.determine:
            next_state = tuple(self.state[i] + self.action_dict[action][i] for i in range(2))
            
            if self.is_within_bounds(next_state) and not next_state in self.obstacles:
                return next_state
            
            return self.state

    # Выводит игровое поле
    def show_board(self):
        self.board[self.state] = 1
        for i in range(BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(BOARD_COLS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                if (i, j) == WIN_STATE:
                    token = 'W'
                if (i, j) == LOSE_STATE:
                    token = 'L'
                out += token + ' | '
            print(out)
        print('-----------------')

## Агент
<font size="3">В результате обучения агент должен уметь для каждого состояния определять лучшее действие. Для этого мы в первую очередь соотнесем каждому состоянию  значение (ожидаемое вознаграждение), используя формулу:</font>
### $V(S_t) \leftarrow V(S_t) + \alpha [V(S_{t+1}) - V(S_t)], \tag{1}$
<font size="3">где $S_t$ - текущее состояние, $S_{t+1}$ - состояние в следующий момент времени, $\alpha$ - скорость обучения.</font>

<font size="3">Имея такое соответствие, агент cможет на каждом шагу выбирать действие в соответствии с его представлениями о максимальном ожидаемом вознаграждении.</font>

<font size="3">В начале агент инициализирует все значения нулем, затем он случайным образом перемещается по полю, пока не доберётся до терминального положения. После этого с помощью формулы (1), проходя по траектории от терминального к начальному состоянию, агент обновляет значения V-функции для всех пройденных состояний.</font>

In [9]:
class Agent:

    def __init__(self):
        self.states = []                                # последовательный набор состояний, траектория
        self.actions = ["up", "down", "left", "right"]  # множество допустимых действий
        self.State = State()                            # текущее состояние
        self.lr = 0.2                                   # learning rate - \alpha 
        self.exp_rate = 0.3                             # exploration rate - вероятность "попробовать пойти не туда" 

        # функция V инициализируется нулями 
        self.state_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.state_values[(i, j)] = 0
                
        self.State.show_board()

    # Выбирает лучшее действие для текущего состояния
    def choose_action(self):
        max_next_reward = 0
        action = "up"

        # Добавляем элемент случайности, чтобы агент не ходил всегда по одному пути
        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            for a in self.actions:
                next_reward = self.state_values[self.State.next_position(a)]
                if next_reward >= max_next_reward:
                    action = a
                    max_next_reward = next_reward
        return action

    # Возвращает следующее состояние, соответствующее действию action
    def take_action(self, action):
        position = self.State.next_position(action)
        return State(state=position)

    # стирает информацию о пройденной агентом траектории и ставит его на стартовую позицию
    def reset(self):
        self.states = []
        self.State = State()

    # В течение нескольких раундов производим обновление значений,
    # соответствующих каждому состоянию
    def play(self, rounds=10):
        i = 0
        while i < rounds:
            # Конец раунда
            if self.State.is_end:
                reward = self.State.give_reward()
                self.state_values[self.State.state] = reward
                # print("Game End Reward", reward)
                
                # Обратное распространение в конце раунда
                for s in reversed(self.states):
                    reward = self.state_values[s] + self.lr * (reward - self.state_values[s])
                    self.state_values[s] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.choose_action()
                # Добавляем состояние к истории
                self.states.append(self.State.next_position(action))
                # print("current position {} action {}".format(self.State.state, action))
                
                # Переходим в следущее состояние, выполняя действие
                self.State = self.take_action(action)
                
                # Смотрим, достигли ли конечного состояния
                self.State.update_end()
                # print("nxt state", self.State.state)
                # print("---------------------")

    # Выводит текущие значения для всех состояний
    def show_values(self):
        for i in range(BOARD_ROWS):
            print('----------------------------------')
            out = '| '
            for j in range(BOARD_COLS):
                out += str(self.state_values[(i, j)]).ljust(6) + ' | '
            print(out)
        print('----------------------------------')

<font size="3">В итоге агент может достигать наибольшей награды в конце игры, выбирая на каждом шагу действие, приводящее к следующему состоянию с наибольшим значением.</font>

In [8]:
ag = Agent()
ag.play(100)
ag.show_values()

-----------------
| 0 | 0 | 0 | W | 
-----------------
| 0 | z | z | L | 
-----------------
| * | 0 | 0 | 0 | 
-----------------
----------------------------------
| 0.993  | 0.996  | 0.998  | 1.0    | 
----------------------------------
| 0.965  | 0      | 0      | -1.0   | 
----------------------------------
| 0.874  | 0.642  | 0.033  | -0.36  | 
----------------------------------
